# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries

import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here

df = pd.read_csv('data/Admission_predict.csv')
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
df.rename(columns=lambda x: x.strip(), inplace=True)
df = df.rename(columns = lambda x: x)

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
df.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
5,6,321,109,3,3.0,4.0,8.20,1,0.75
6,7,308,101,2,3.0,4.0,7.90,0,0.68
7,8,302,102,1,2.0,1.5,8.00,0,0.50
8,9,323,108,3,3.5,3.0,8.60,0,0.45
9,10,327,111,4,4.0,4.5,9.00,1,0.84


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here

df.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [7]:
# your code here
df = df.set_index('Serial No.')
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [8]:
df['GRE CGPA'] = df['GRE Score'].astype(str) + '_' + df['CGPA'].astype(str)

# combination of both columns

In [9]:
df['GRE CGPA'].nunique()

385

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
# your code here



# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [10]:
# your code here

df.loc[(df['CGPA'] > 9) & (df['Research'] == 1)]

# 101 rows

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE CGPA
Serial No.,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34
11,328,112,4,4.0,4.5,9.10,1,0.78,328_9.1
20,328,116,5,5.0,5.0,9.50,1,0.94,328_9.5
21,334,119,5,5.0,4.5,9.70,1,0.95,334_9.7
...,...,...,...,...,...,...,...,...,...
380,329,111,4,4.5,4.0,9.23,1,0.89,329_9.23
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# your code here

df.loc[(df['CGPA'] > 9) & (df['SOP'] < 3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    
    mean = np.mean(col)
    std = np.std(col)
    
    return [(element - mean)/std for element in col]

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [13]:
# your code here

df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR'])

In [14]:
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE CGPA,CGPA_std,GRE_std,LOR_std
Serial No.,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65,1.750174,1.755663,1.193197
2,316,104,3,3.0,3.5,8.00,1,0.72,316_8.0,-0.992501,-0.063450,0.076840
3,322,110,3,3.5,2.5,8.67,1,0.80,322_8.67,0.121191,0.456297,-1.039517
4,314,103,2,2.0,3.0,8.21,0,0.65,314_8.21,-0.643433,-0.236698,-0.481338
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04,0.736216,0.629546,0.076840
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11,0.852571,0.716170,0.076840
383,330,116,4,5.0,4.5,9.45,1,0.91,330_9.45,1.417729,1.149292,1.193197


We will generate the decision choice at random using the code below. Please run the cell.

In [15]:
# Libraries
from random import choices

In [16]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [17]:
# your code here
df['deciding_column'] = decision_choice
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE CGPA,CGPA_std,GRE_std,LOR_std,deciding_column
Serial No.,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65,1.750174,1.755663,1.193197,GRE_std
2,316,104,3,3.0,3.5,8.00,1,0.72,316_8.0,-0.992501,-0.063450,0.076840,LOR_std
3,322,110,3,3.5,2.5,8.67,1,0.80,322_8.67,0.121191,0.456297,-1.039517,CGPA_std
4,314,103,2,2.0,3.0,8.21,0,0.65,314_8.21,-0.643433,-0.236698,-0.481338,LOR_std
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34,1.234884,1.149292,-0.481338,CGPA_std
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04,0.736216,0.629546,0.076840,CGPA_std
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11,0.852571,0.716170,0.076840,LOR_std
383,330,116,4,5.0,4.5,9.45,1,0.91,330_9.45,1.417729,1.149292,1.193197,GRE_std


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [18]:
# your code here
df['deciding_column_2'] = [b if a == 'CGPA_std' else c if a == 'GRE_std' else d for a,b,c,d in zip(df['deciding_column'],df['CGPA_std'],df['GRE_std'],df['LOR_std'])]
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE CGPA,CGPA_std,GRE_std,LOR_std,deciding_column,deciding_column_2
Serial No.,,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65,1.750174,1.755663,1.193197,GRE_std,1.755663
2,316,104,3,3.0,3.5,8.00,1,0.72,316_8.0,-0.992501,-0.063450,0.076840,LOR_std,0.076840
3,322,110,3,3.5,2.5,8.67,1,0.80,322_8.67,0.121191,0.456297,-1.039517,CGPA_std,0.121191
4,314,103,2,2.0,3.0,8.21,0,0.65,314_8.21,-0.643433,-0.236698,-0.481338,LOR_std,-0.481338
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34,1.234884,1.149292,-0.481338,CGPA_std,1.234884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04,0.736216,0.629546,0.076840,CGPA_std,0.736216
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11,0.852571,0.716170,0.076840,LOR_std,0.076840
383,330,116,4,5.0,4.5,9.45,1,0.91,330_9.45,1.417729,1.149292,1.193197,GRE_std,1.149292


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [21]:
# your code here

df['decision'] = [1 if element > 0.8 else 0 for element in df['deciding_column_2']]
df['decision'].value_counts()

# 83 applicants would be accepted into the program.

0    302
1     83
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [22]:
# your code here

df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
df = df.rename(columns = lambda x: x)

In [23]:
df

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,gre_cgpa,cgpa_std,gre_std,lor_std,deciding_column,deciding_column_2,decision
Serial No.,,,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65,1.750174,1.755663,1.193197,GRE_std,1.755663,1
2,316,104,3,3.0,3.5,8.00,1,0.72,316_8.0,-0.992501,-0.063450,0.076840,LOR_std,0.076840,0
3,322,110,3,3.5,2.5,8.67,1,0.80,322_8.67,0.121191,0.456297,-1.039517,CGPA_std,0.121191,0
4,314,103,2,2.0,3.0,8.21,0,0.65,314_8.21,-0.643433,-0.236698,-0.481338,LOR_std,-0.481338,0
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34,1.234884,1.149292,-0.481338,CGPA_std,1.234884,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04,0.736216,0.629546,0.076840,CGPA_std,0.736216,0
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11,0.852571,0.716170,0.076840,LOR_std,0.076840,0
383,330,116,4,5.0,4.5,9.45,1,0.91,330_9.45,1.417729,1.149292,1.193197,GRE_std,1.149292,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [24]:
# your code here

df['adjusted_gre'] = [a + 10 if b >= 4 else a for (a,b) in zip(df['gre_score'],df['university_rating'])]
df

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,gre_cgpa,cgpa_std,gre_std,lor_std,deciding_column,deciding_column_2,decision,adjusted_gre
Serial No.,,,,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,337_9.65,1.750174,1.755663,1.193197,GRE_std,1.755663,1,347
2,316,104,3,3.0,3.5,8.00,1,0.72,316_8.0,-0.992501,-0.063450,0.076840,LOR_std,0.076840,0,316
3,322,110,3,3.5,2.5,8.67,1,0.80,322_8.67,0.121191,0.456297,-1.039517,CGPA_std,0.121191,0,322
4,314,103,2,2.0,3.0,8.21,0,0.65,314_8.21,-0.643433,-0.236698,-0.481338,LOR_std,-0.481338,0,314
5,330,115,5,4.5,3.0,9.34,1,0.90,330_9.34,1.234884,1.149292,-0.481338,CGPA_std,1.234884,1,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,324_9.04,0.736216,0.629546,0.076840,CGPA_std,0.736216,0,324
382,325,107,3,3.0,3.5,9.11,1,0.84,325_9.11,0.852571,0.716170,0.076840,LOR_std,0.076840,0,325
383,330,116,4,5.0,4.5,9.45,1,0.91,330_9.45,1.417729,1.149292,1.193197,GRE_std,1.149292,1,340


In [25]:
pd.cut(df['adjusted_gre'], bins = 4)

Serial No.
1      (335.0, 350.0]
2      (305.0, 320.0]
3      (320.0, 335.0]
4      (305.0, 320.0]
5      (335.0, 350.0]
            ...      
381    (320.0, 335.0]
382    (320.0, 335.0]
383    (335.0, 350.0]
384    (305.0, 320.0]
385    (335.0, 350.0]
Name: adjusted_gre, Length: 385, dtype: category
Categories (4, interval[float64]): [(289.94, 305.0] < (305.0, 320.0] < (320.0, 335.0] < (335.0, 350.0]]